In [95]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import optuna
from sklearn.model_selection import train_test_split,cross_val_score
import xgboost as xgb

In [107]:
trainData=pd.read_csv('housePrice/train.csv')
traindf=pd.DataFrame(trainData)

testData=pd.read_csv('housePrice/test.csv')
testdf=pd.DataFrame(trainData)

testdf.head(10)


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,...,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,...,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,NaN,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,...,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,...,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,NaN,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,...,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,...,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000
5,6,50,RL,85.0,14115,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Mitchel,Norm,Norm,1Fam,1.5Fin,5,5,1993,1995,Gable,CompShg,VinylSd,VinylSd,NaN,0.0,TA,TA,Wood,Gd,TA,No,GLQ,732,Unf,0,64,796,GasA,...,Y,SBrkr,796,566,0,1362,1,0,1,1,1,1,TA,5,Typ,0,NaN,Attchd,1993.0,Unf,2,480,TA,TA,Y,40,30,0,320,0,0,NaN,MnPrv,Shed,700,10,2009,WD,Normal,143000
6,7,20,RL,75.0,10084,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Somerst,Norm,Norm,1Fam,1Story,8,5,2004,2005,Gable,CompShg,VinylSd,VinylSd,Stone,186.0,Gd,TA,PConc,Ex,TA,Av,GLQ,1369,Unf,0,317,1686,GasA,...,Y,SBrkr,1694,0,0,1694,1,0,2,0,3,1,Gd,7,Typ,1,Gd,Attchd,2004.0,RFn,2,636,TA,TA,Y,255,57,0,0,0,0,NaN,NaN,NaN,0,8,2007,WD,Normal,307000
7,8,60,RL,NaN,10382,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,NWAmes,PosN,Norm,1Fam,2Story,7,6,1973,1973,Gable,CompShg,HdBoard,HdBoard,Stone,240.0,TA,TA,CBlock,Gd,TA,Mn,ALQ,859,BLQ,32,216,1107,GasA,...,Y,SBrkr,1107,983,0,2090,1,0,2,1,3,1,TA,7,Typ,2,TA,Attchd,1973.0,RFn,2,484,TA,TA,Y,235,204,228,0,0,0,NaN,NaN,Shed,350,11,2009,WD,Normal,200000
8,9,50,RM,51.0,6120,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,OldTown,Artery,Norm,1Fam,1.5Fin,7,5,1931,1950,Gable,CompShg,BrkFace,Wd Shng,NaN,0.0,TA,TA,BrkTil,TA,TA,No,Unf,0,Unf,0,952,952,GasA,...,Y,FuseF,1022,752,0,1774,0,0,2,0,2,2,TA,8,Min1,2,TA,Detchd,1931.0,Unf,2,468,Fa,TA,Y,90,0,205,0,0,0,NaN,NaN,NaN,0,4,2008,WD,Abnorml,129900
9,10,190,RL,50.0,7420,Pave,NaN,Reg,Lvl,AllPub,Corner,Gtl,BrkSide,Artery,Artery,2fmCon,1.5Unf,5,6,1939,1950,Gable,CompShg,MetalSd,MetalSd,NaN,0.0,TA,TA,BrkTil,TA,TA,No,GLQ,851,Unf,0,140,991,GasA,

In [143]:
testdf.shape

(1460, 81)

In [53]:
traindf2=traindf.copy()
testdf2=testdf.copy()

In [55]:
traindf2.drop(['Alley'],axis='columns',inplace=True)
testdf2.drop(['Alley'],axis='columns',inplace=True)

In [142]:
testdf2.shape

(1460, 80)

In [ ]:
traindf2.fillna({
    col: traindf2[col].mean() if traindf2[col].dtype in ['int64', 'float64'] else traindf2[col].mode()[0]
    for col in traindf2.columns
}, inplace=True)
testdf2.fillna({
    col: testdf2[col].mean() if testdf2[col].dtype in ['int64', 'float64'] else testdf2[col].mode()[0]
    for col in testdf2.columns
}, inplace=True)

In [86]:
traindf2.isnull().sum().head(50)

Id              0
MSSubClass      0
MSZoning        0
LotFrontage     0
LotArea         0
Street          0
LotShape        0
LandContour     0
Utilities       0
LotConfig       0
LandSlope       0
Neighborhood    0
Condition1      0
Condition2      0
BldgType        0
HouseStyle      0
OverallQual     0
OverallCond     0
YearBuilt       0
YearRemodAdd    0
RoofStyle       0
RoofMatl        0
Exterior1st     0
Exterior2nd     0
MasVnrType      0
MasVnrArea      0
ExterQual       0
ExterCond       0
Foundation      0
BsmtQual        0
BsmtCond        0
BsmtExposure    0
BsmtFinType1    0
BsmtFinSF1      0
BsmtFinType2    0
BsmtFinSF2      0
BsmtUnfSF       0
TotalBsmtSF     0
Heating         0
HeatingQC       0
CentralAir      0
Electrical      0
1stFlrSF        0
2ndFlrSF        0
LowQualFinSF    0
GrLivArea       0
BsmtFullBath    0
BsmtHalfBath    0
FullBath        0
HalfBath        0
dtype: int64

In [89]:
testdf2.isnull().sum().head(61)


Id              0
MSSubClass      0
MSZoning        0
LotFrontage     0
LotArea         0
               ..
FireplaceQu     0
GarageType      0
GarageYrBlt     0
GarageFinish    0
GarageCars      0
Length: 61, dtype: int64

In [90]:
for col in traindf2.select_dtypes(include=['object']).columns:
    le = LabelEncoder()
    traindf2[col] = le.fit_transform(traindf2[col])

In [91]:
for col in testdf2.select_dtypes(include=['object']).columns:
    le = LabelEncoder()
    testdf2[col] = le.fit_transform(testdf2[col])

In [97]:
X=traindf2.drop(['SalePrice'],axis='columns')
Y=traindf2['SalePrice']

In [100]:


def objective(trial):
    
    n_estimators = trial.suggest_int("n_estimators", 10, 100, step=10)
    learning_rate = trial.suggest_float("learning_rate", 0.01, 0.3, step=0.01) 
    max_depth = trial.suggest_categorical("max_depth", [5, 10, 15, None])
    subsample = trial.suggest_float("subsample", 0.5, 1.0, step=0.1)  
    colsample_bytree = trial.suggest_float("colsample_bytree", 0.5, 1.0, step=0.1) 



    model = xgb.XGBRegressor(
        n_estimators=n_estimators,
        learning_rate=learning_rate,
        max_depth=max_depth,
        subsample=subsample,
        colsample_bytree=colsample_bytree,
        random_state=42  
    )

 
    score = cross_val_score(model, X, Y, cv=3, scoring='neg_mean_squared_error').mean()
    
    return score  


In [101]:
study=optuna.create_study(direction='maximize',sampler=optuna.samplers.TPESampler())


[I 2025-02-17 18:52:37,488] A new study created in memory with name: no-name-53080f15-2501-426f-9b1d-58d3d6222174


In [102]:
study.optimize(objective,n_trials=50)

[I 2025-02-17 18:52:41,187] Trial 0 finished with value: -983911962.3331381 and parameters: {'n_estimators': 30, 'learning_rate': 0.3, 'max_depth': 15, 'subsample': 0.7, 'colsample_bytree': 0.6}. Best is trial 0 with value: -983911962.3331381.
[I 2025-02-17 18:52:42,990] Trial 1 finished with value: -877803356.2549144 and parameters: {'n_estimators': 60, 'learning_rate': 0.11, 'max_depth': 10, 'subsample': 0.5, 'colsample_bytree': 0.5}. Best is trial 1 with value: -877803356.2549144.
[I 2025-02-17 18:52:44,306] Trial 2 finished with value: -915370011.1891985 and parameters: {'n_estimators': 60, 'learning_rate': 0.05, 'max_depth': None, 'subsample': 0.7, 'colsample_bytree': 0.9}. Best is trial 1 with value: -877803356.2549144.
[I 2025-02-17 18:52:45,508] Trial 3 finished with value: -807848924.2327085 and parameters: {'n_estimators': 60, 'learning_rate': 0.14, 'max_depth': None, 'subsample': 0.7, 'colsample_bytree': 0.9}. Best is trial 3 with value: -807848924.2327085.
[I 2025-02-17 18:

In [103]:
print(f"Best Accuracy is: {study.best_trial.value}\nBest Parameters are: {study.best_trial.params}")

Best Accuracy is: -701143891.0660368
Best Parameters are: {'n_estimators': 80, 'learning_rate': 0.12, 'max_depth': 5, 'subsample': 0.6, 'colsample_bytree': 0.7}


In [104]:
PridectModel=xgb.XGBRegressor(**study.best_trial.params)
PridectModel.fit(X,Y)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.7, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.12, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=5, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=80, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [110]:
X_test=testdf2.drop(['SalePrice'],axis='columns')

In [111]:
predicted=PridectModel.predict(X_test)
predicted

array([200591.92, 174279.48, 210951.72, ..., 258852.97, 135321.61,
       147555.08], dtype=float32)

In [144]:
submission1=pd.DataFrame({'Id':testdf2['Id'],'SalePrice':predicted})

In [145]:
submission1 = submission1.iloc[:-1]

In [146]:
submission1.to_csv('submission1.csv',index=False)

In [147]:
submission1.shape

(1459, 2)

In [141]:
submission1.head(10)

,Id,SalePrice
0,1,200591.921875
1,2,174279.484375
2,3,210951.718750
3,4,151019.500000
4,5,272114.375000
5,6,154980.375000
6,7,295624.375000
7,8,213830.859375
8,9,131630.125000
9,10,122913.765625
